In [97]:
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

In [110]:
X_train=pd.read_csv('X_train.csv', index_col=0)

In [111]:
X_train.head()

,time,seconds_elapsed_orientation,qz_orientation,qy_orientation,qx_orientation,qw_orientation,roll_orientation,pitch_orientation,yaw_orientation,seconds_elapsed_location,...,horizontalAccuracy_location_network,speed_location_network,bearing_location_network,altitude_location_network,longitude_location_network,latitude_location_network,seconds_elapsed_gravity,z_gravity,y_gravity,x_gravity
1532440,1694856436817851600,127.575852,-0.659162,0.252138,-0.704714,0.072871,-1.704910,0.450156,-2.849463,127.072000,...,122.400002,0.0,0.0,15.700000,3.100094,50.682566,127.575852,-1.180672,-4.266960,8.750398
1241906,1694850695837889000,508.709889,0.380207,-0.870947,0.300362,0.081716,-2.653062,0.660095,-0.834548,508.191000,...,800.000000,0.0,0.0,0.000000,3.123176,50.672179,508.709889,-6.836542,-6.013351,3.643037
57903,1694672680729749800,129.277750,0.528210,-0.454058,-0.605905,0.384324,2.037298,1.238830,2.683162,128.796625,...,34.575001,0.0,0.0,89.000000,3.139532,50.685145,129.277750,-1.437885,-9.271173,-2.854510
2279316,1694888562008397000,481.953397,0.162305,-0.127510,-0.579647,0.788294,-0.043528,1.270524,-0.374022,481.764502,...,35.208000,0.0,0.0,88.900002,3.148445,50.684358,481.953397,2.910758,-9.364353,0.082350
520841,1694680852518048500,187.191049,0.305719,-0.296312,0.892543,0.148668,-2.452161,-0.084309,-2.470231,186.989000,...,14.507000,0.0,0.0,68.599998,3.095495,50.688239,187.191049,-7.539892,0.825681,6.216005


In [131]:
class AverageSecondsElapsedTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, keyword='seconds_elapsed_'):
        self.keyword = keyword
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        columns_to_average = X.filter(like=self.keyword).columns
        X['seconds_elapsed_time'] = X[columns_to_average].mean(axis=1)
        X = X.drop(columns=columns_to_average)
        return X

In [133]:
class ColumnDropper(BaseEstimator, TransformerMixin):
    def __init__(self, columns=None):
        self.columns = columns
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        if self.columns is not None:
            X = X.drop(columns=self.columns, errors='ignore')
        return X

In [134]:
pipeline = Pipeline(steps=[
    ('dropper', ColumnDropper(columns=["bearingAccuracy_location_network", 
                                        "speedAccuracy_location_network", 
                                        "speed_location_network", 
                                        "bearing_location_network"])),
    ('average_seconds_elapsed', AverageSecondsElapsedTransformer())
])

In [135]:
X_train_transformed = pipeline.fit_transform(X_train)

In [122]:
X_train_transformed.head()

,time,qz_orientation,qy_orientation,qx_orientation,qw_orientation,roll_orientation,pitch_orientation,yaw_orientation,bearingAccuracy_location,speedAccuracy_location,...,horizontalAccuracy_location_network,speed_location_network,bearing_location_network,altitude_location_network,longitude_location_network,latitude_location_network,z_gravity,y_gravity,x_gravity,seconds_elapsed_time
1532440,1694856436817851600,-0.659162,0.252138,-0.704714,0.072871,-1.704910,0.450156,-2.849463,8.600000,1.5,...,122.400002,0.0,0.0,15.700000,3.100094,50.682566,-1.180672,-4.266960,8.750398,126.463609
1241906,1694850695837889000,0.380207,-0.870947,0.300362,0.081716,-2.653062,0.660095,-0.834548,14.100000,1.7,...,800.000000,0.0,0.0,0.000000,3.123176,50.672179,-6.836542,-6.013351,3.643037,471.666120
57903,1694672680729749800,0.528210,-0.454058,-0.605905,0.384324,2.037298,1.238830,2.683162,0.000000,0.0,...,34.575001,0.0,0.0,89.000000,3.139532,50.685145,-1.437885,-9.271173,-2.854510,121.630446
2279316,1694888562008397000,0.162305,-0.127510,-0.579647,0.788294,-0.043528,1.270524,-0.374022,0.000000,0.0,...,35.208000,0.0,0.0,88.900002,3.148445,50.684358,2.910758,-9.364353,0.082350,481.079536
520841,1694680852518048500,0.305719,-0.296312,0.892543,0.148668,-2.452161,-0.084309,-2.470231,179.899994,2.6,...,14.507000,0.0,0.0,68.599998,3.095495,50.688239,-7.539892,0.825681,6.216005,161.313576


In [140]:
len(X_train_transformed.columns)

48